# Sentiment analysis using Amazon product reviews

### Dataset: Datafiniti Consumer Reviews of Amazon Products

##### The file sentiment_analysis.ipynb represents the sentiment analysis using Python functions. The sentiment analysis is focused on the calculation of the sentiment measures and the representation of results in a useful for the user way.
##### The file represents a Jupyter Notebook and contains several blocks: import of libraries, description of functions, and graphical representation of results.
##### The dataset Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv is sourced from [kaggle.com] (https://www.kaggle.com/datasets/datafiniti/consumer-reviews-of-amazon-products) and represents a CSV file.
##### The CSV dataset is preprocessed using dropna and stop_words functions.
##### The sentiment analysis uses the polarity function and calculates cumulative polarity for each sentence and the whole dataset to represent the final scores.
##### The positive and negative words in the dataset help to represent the results using TextBlob and WordCloud.
##### The analysis also calculates the standard deviation to check the statistical meaning of the result and concludes the positive sentiment of the dataset.
##### The installation and usage of the file will require Jupyther Notebook and the installation of libraries such as Pandas, Numpy, SpaCy, TextBlob.

In [3]:
# Import libraries

import pandas as pd
import numpy as np
import spacy
from textblob import TextBlob


In [6]:
# Reading data 

data = pd.read_csv('amazon_product_reviews.csv')

### We will use spacy model to analyse the dataset

In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
# Create a dataframe with cumulative polarity column
reviews_data = pd.DataFrame(data['reviews.text'])
reviews_data.dropna(inplace=True)
reviews_data.columns = ["Sentence"] 
reviews_data["Cumulative Polarity"] = None
reviews_data.head()


,Sentence,Cumulative Polarity
0,I thought it would be as big as small paper bu...,None
1,This kindle is light and easy to use especiall...,None
2,Didnt know how much i'd use a kindle so went f...,None
3,I am 100 happy with my purchase. I caught it o...,None
4,Solid entry level Kindle. Great for kids. Gift...,None


In [9]:
# Functions which help to assign a cumulative polarity to all words in a sentence

def cumulative_polarity(sentence: str) -> float:
    
    # Argument validation
    if isinstance (sentence, str) is False:
        raise TypeError("The argument `sentence` must be of type string.")
    # Compute the polarity of every word in a sentence and add up all polarities
    cumulative_polarity = 0.0
    words = sentence.lower().split()
    for word in words:
        if word in nlp.Defaults.stop_words:
            continue
        blob = TextBlob(word)
        cumulative_polarity += blob.sentiment.polarity
    return cumulative_polarity



def main(df: pd.DataFrame) -> None: 
    # Arguments validation
    if isinstance(df,pd.DataFrame) is False:
        raise TypeError("The argument df must be of pandas.DataFrame type.")
    # Compute polarity for every sentence
    for i in range(len(df)):
        sentence = df.iloc[i, 0]
        df.iloc[i, 1] = cumulative_polarity(sentence)

        

In [10]:

main(df=reviews_data)

In [12]:
# Assignsentiment based on polarity score

polarity_score = reviews_data['Cumulative Polarity'].mean()
if polarity_score > 0:
    sentiment = 'positive'
elif polarity_score < 0:
    sentiment = 'negative'
else:
    sentiment = 'neutral'

print(f"Reviews' Mean polarity score: {polarity_score}\nSentiment: {sentiment}")


Reviews' Mean polarity score: 1.0928375970279833
Sentiment: positive


In [13]:
stdev = reviews_data['Cumulative Polarity'].std()
print(f" Standard deviation of the polarity is {stdev}.")

 Standard deviation of the polarity is 1.0253841916109658.


In [14]:
# Sample polarity by sentence
sample = reviews_data.head()
main(df=reviews_data)
print (sample)

                                            Sentence Cumulative Polarity
0  I thought it would be as big as small paper bu...                -0.1
1  This kindle is light and easy to use especiall...            0.833333
2  Didnt know how much i'd use a kindle so went f...              0.4625
3  I am 100 happy with my purchase. I caught it o...               4.125
4  Solid entry level Kindle. Great for kids. Gift...                3.25


### Use wordcloud to chart positive and negative words

In [15]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from collections import defaultdict

In [16]:
# Initialize dictionaries to hold positive and negative words
positive_words = defaultdict(int)
negative_words = defaultdict(int)

In [17]:

def words_sent(df: pd.DataFrame) -> None:
    if isinstance(df,pd.DataFrame) is False:
        raise TypeError
    for i in range(len(df)):
        sentence = df.iloc[i,0]
        
           # for words in sentences:
        words = sentence.split()

        for word in words:
            blob = TextBlob(word)
            polarity = blob.sentiment.polarity

            if polarity > 0:
                positive_words[word.lower()] += 1
            elif polarity < 0:
                negative_words[word.lower()] += 1

In [18]:
words_sent(reviews_data)
pos_wordcloud = WordCloud(width=400, height=200, background_color ='white').generate_from_frequencies(positive_words)
neg_wordcloud = WordCloud(width=400, height=200, background_color ='white').generate_from_frequencies(negative_words)

In [19]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(pos_wordcloud, interpolation='bilinear')
ax[0].set_title('Positive Words')
ax[0].axis('off')

ax[1].imshow(neg_wordcloud, interpolation='bilinear')
ax[1].set_title('Negative Words')
ax[1].axis('off')

plt.show()

### The Amazon product review dataset provides a consistent picture of positive reviews. The negative word cloud includes some qualitative characteristics which are rather neutral like 'small','previously','long','other','little' etc. The mean polatity score is 1.09 and the claculated sentiment is Positive.